In [ ]:
from google.colab import drive
import sys

drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/My Drive/IOT_DL_project/')

Mounted at /content/gdrive/


In [ ]:
%cd '/content/gdrive/My Drive/IOT_DL_project/dataset/1.Bot-IoT/train_test_only_five_percent/All_features'

/content/gdrive/.shortcut-targets-by-id/1iU-5odoNw4f2D_AgPNxVDuu6YX9juYYv/IOT_DL_project/dataset/1.Bot-IoT/train_test_only_five_percent/All_features


In [ ]:
import numpy as np
import sklearn
import math
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import time
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical 

### Attack vs Non-Attack

In [ ]:
df=pd.read_csv("UNSW_2018_IoT_Botnet_Full5pc_4.csv")
cols=list(df.columns.values)

arr=pd.unique(df['state'])
dict={'RST':1, 'CON':2, 'REQ':3, 'INT':4, 'URP':5}
df['state']=df['state'].map(dict)
arr2=pd.unique(df['flgs'])
arr3=pd.unique(df['proto'])
dict2={'e':1, 'e s':2, 'e d':3, 'e *':4, 'e g':5, 'eU':6}
df['flgs']=df['flgs'].map(dict2)
dict3={'tcp':1, 'arp':2, 'udp':3, 'icmp':4}
df['proto']=df['proto'].map(dict3)


df.drop('stime',inplace=True,axis=1)
df.drop('ltime',inplace=True,axis=1)
df.drop('pkSeqID',inplace=True,axis=1)
df.drop('category',inplace=True,axis=1)
df.drop('subcategory',inplace=True,axis=1)
df.drop('dport',inplace=True,axis=1)
df.drop('sport',inplace=True,axis=1)

arr4=pd.unique(df['saddr'])
dict4={'192.168.100.147':1, '192.168.100.7':2, '192.168.100.148':3, '192.168.100.6':4,
 '192.168.100.149':5, '192.168.100.150':6, '192.168.100.5':7, '192.168.100.3':8}
df['saddr']=df['saddr'].map(dict4)
df['daddr']=df['daddr'].map(dict4)

df=df.dropna()

x,y=df.values[:,:-1],df.values[:,-1:]
x=StandardScaler().fit_transform(x)

sm=SMOTE(k_neighbors=6)
x,y=sm.fit_resample(x,y)

x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=12)
x_val,x_test,y_val,y_test=train_test_split(x_val,y_val,test_size=0.5,random_state=12)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1058508, 38)
(132314, 38)
(1058508,)
(132314,)


In [ ]:
class TriTrainingwDisagreement():

    def __init__(self, classifier):
        """
        args:
            classifier - classifier, with .fit, .predict API (refer to classifiers of sklearn)
        """
        # Initialize
        if sklearn.base.is_classifier(classifier):
            self.clf = [sklearn.base.clone(classifier) for i in range(3)]
        else:
            self.clf = [sklearn.base.clone(classifier[i]) for i in range(3)]

    def measure_error(self, j, k):
        """
        args:
                j - int, classifier index
                k - int, classifier index
        return:
                float, classification_error
        """
        y_predict_j = self.clf[j].predict(self.X_label)
        y_predict_k = self.clf[k].predict(self.X_label)
        return (1 - np.sum((y_predict_j == y_predict_k) & (y_predict_j == self.y_label)) / np.sum(y_predict_j == y_predict_k))

    def fit(self, X_label, y_label, X_unlabel):
        """
        args:
                X_label - labeled train feature vector (ndarray of size, # of samples * # of features), features are numeric numbers
                y_label - labeled train label vector (ndarray of size, # of samples), labels are numeric numbers
                X_unlabel - test feature vector (ndarray of size, # of samples * # of features), features are numeric numbers
        """        

        self.X_label = X_label
        self.y_label = y_label

        classification_error_current = [0.5, 0.5, 0.5]
        classification_error = [0.5, 0.5, 0.5]
        pseudo_label_size_current = [0, 0, 0]
        pseudo_label_size = [0, 0, 0]
        # pseudo_label_index used to compare and check if tri-training can be stopped, when two iterations have the same label_index, means tri-training can be stopped
        X_pseudo_label_index = [[], [], []]
        X_pseudo_label_index_current = [[], [], []]

        feature_size = self.X_label.shape[1]

        # Train each classifier with bootstrampped subset
        for i in range(3):
            X_resample, y_resample = sklearn.utils.resample(self.X_label, self.y_label)  # BootstrapSample(L)
            self.clf[i].fit(X_resample, y_resample)  # Learn(Si)

        iteration = 0
        while (True):

            update = [False, False, False]

            iteration = iteration + 1
            for i in range(3):
                X_pseudo_label_index_current[i] = X_pseudo_label_index[i]

            # Step3.1 Set Li = empty set, Li denotes the new pseudo label set determined by tri-training iteration for classifier i
            # X_pseudo_label_index, contains the data record index (in the full unlabelled set) of the new pseudo label set determined by tri-training iteration for classifier i
            # X_pseudo_label, contains the features for new pseudo label set determined by tri-training iteration for classifier i
            # y_pseudo_label, contains the labels (not ground truth label, but pseudo label calculated by tri-training iteration) for new pseudo label set determined by tri-training iteration for classifier i
            X_pseudo_label_index = [[], [], []]
            X_pseudo_label = [[], [], []]
            y_pseudo_label = [[], [], []]

            # Step 3.2 Loop through all the data record in unlabelled set
            for i in range(3):
                j, k = np.delete(np.array([0, 1, 2]), i)
                classification_error[i] = self.measure_error(j, k)
                if classification_error[i] < classification_error_current[i]:
                    # Step 3.2 If classifier j,k aggrees with the label for one data record, and not agree with classifier i, in unlabelled set,
                    # then add the data record into Li                    
                    y_predict_j = self.clf[j].predict(X_unlabel)
                    y_predict_k = self.clf[k].predict(X_unlabel)
                    y_predict_i = self.clf[i].predict(X_unlabel)
                    y_pseudo_label[i] = y_predict_j[np.logical_and(y_predict_j==y_predict_k,y_predict_j!=y_predict_i)]
                    X_pseudo_label_index[i] = np.where(np.logical_and(y_predict_j==y_predict_k,y_predict_j!=y_predict_i))
                    
                    pseudo_label_size[i] = len(X_pseudo_label_index[i])
                    #print("classification_error: {}, classification_error_current: {}, pseudo_label_size: {}, pseudo_label_size_current: {}".format(classification_error[i], classification_error_current[i], pseudo_label_size[i],pseudo_label_size_current[i]))

                    if pseudo_label_size_current[i] == 0:
                        pseudo_label_size_current[i] = math.floor(classification_error[i] / (classification_error_current[i] - classification_error[i]) + 1)
                    if pseudo_label_size_current[i] < pseudo_label_size[i]:
                        if ((classification_error[i] * pseudo_label_size[i]) < (classification_error_current[i] * pseudo_label_size_current[i])):
                            update[i] = True
                        elif pseudo_label_size_current[i] > (classification_error[i] / (classification_error_current[i] - classification_error[i])):
                            resample_size = math.ceil(classification_error_current[i] * pseudo_label_size_current[i] / classification_error[i] - 1)
                            X_pseudo_label_index[i], y_pseudo_label[i] = sklearn.utils.resample(X_pseudo_label_index[i],y_pseudo_label[i],replace=False,n_samples=resample_size)
                            pseudo_label_size[i] = len(X_pseudo_label_index[i])
                            update[i] = True

            # Step 3.3 Train all the three classifiers with Li + original labelled data set
            for i in range(3):
                if update[i] == True:
                    #print("number of pseudo labels added for classifier {} is: {}".format(i,len(X_pseudo_label_index[i])))
                    X_pseudo_label[i] = np.array(X_unlabel[X_pseudo_label_index[i]])
                    self.clf[i].fit(np.concatenate((X_pseudo_label[i], self.X_label), axis=0),np.concatenate((np.array(y_pseudo_label[i]), self.y_label), axis=0))
                    classification_error_current[i] = classification_error[i]
                    pseudo_label_size_current[i] = pseudo_label_size[i]

            # Stop tri-training process, if the pseudo label data set added in current tri-training iteration
            # is the same for last tri-training iteration for all classifiers
            if (np.array_equal(X_pseudo_label_index[0], X_pseudo_label_index_current[0]) & np.array_equal(X_pseudo_label_index[1], X_pseudo_label_index_current[1]) 
                    & np.array_equal(X_pseudo_label_index[2], X_pseudo_label_index_current[2])):
                break

    def predict(self, X_test):
        """
        args:
                X_test - test feature vector (ndarray of size, # of samples * # of features), features are numeric numbers
        return:
                array of size (# of test samples), with values as predicted label 1 or 0
        """
        I = self.clf[0].predict(X_test)
        J = self.clf[1].predict(X_test)
        K = self.clf[2].predict(X_test)
        I[J == K] = J[J == K]
        return I

    def score(self, X_test, y_test):
        """ 
        args:
                X_test - test feature vector (ndarray of size, # of samples * # of features), features are numeric numbers
                y_test - test label vector (ndarray of size, # of samples), labels are numeric numbers
        return:
                float, accuracy_score of predicted value by the tri-training (with disagreement) classifier against groud truth
        """
        
        return sklearn.metrics.accuracy_score(y_test, self.predict(x_test))
    def prec(self, X_test, y_test):
        """ 
        args:
                X_test - test feature vector (ndarray of size, # of samples * # of features), features are numeric numbers
                y_test - test label vector (ndarray of size, # of samples), labels are numeric numbers
        return:
                float, accuracy_score of predicted value by the tri-training (with disagreement) classifier against groud truth
        """
        
        return sklearn.metrics.precision_score(y_test, self.predict(x_test))
    def recall(self, X_test, y_test):
        return sklearn.metrics.recall_score(y_test, self.predict(x_test))
    def F1(self, X_test, y_test):
        return sklearn.metrics.f1_score(y_test, self.predict(x_test))


In [ ]:
def data_process(X_train, y_train, rate):
  rng = np.random.RandomState(0)#to make same index every time
  labeled_index = rng.rand(len(y_train)) < rate#in training set, choose 20% as labeled data
  unlabeled_index = np.logical_not(labeled_index)
  L_data = X_train[labeled_index]#data of L
  L_label = y_train[labeled_index]#lable of L
  U_data = X_train[unlabeled_index]#data of U
  return L_data, L_label, U_data

In [ ]:
classifier = {}

from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn import linear_model
from sklearn import svm

classifier['DecisionTree'] = tree.DecisionTreeClassifier()
classifier['BP_Network'] = MLPClassifier(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(20, 10), random_state=1)
classifier['NaiveBayes'] = GaussianNB()


In [ ]:
L_data, L_label, U_data, X_test, y_test = data_process(x,y, 0.25)

classifiers = [sklearn.base.clone(classifier[c])]
for clf in classifier.keys():
    if clf != c:
        print(clf)
        classifiers.append(sklearn.base.clone(classifier[clf]))#use the first clf in classifiers to output score, but all three to label data
print(classifiers)
m = TriTraining(classifiers)
1 - m.train(L_data, L_label, U_data, X_test, y_test)

In [ ]:
for c in classifier:
        for r in [0.2]:        
            
            print('classifier:', c)
            print('label_rate:', r)
            error = np.zeros([1,20])
            accuracy = np.zeros([1,20])
            precision = np.zeros([1,20])
            recall = np.zeros([1,20])
            f1 = np.zeros([1,20])
            time_model = np.zeros([1,20])
            for i in range(20):#average on 20 data splits
                L_data, L_label, U_data = data_process(x_train, y_train, r)
               
                m2 = TriTrainingwDisagreement(classifier[c])
                start = time.time()
                m2.fit(L_data, L_label, U_data)
                stop = time.time()
                error[0, i] = 1-m2.score(x_test, y_test)
                accuracy[0,i]=m2.score(x_test,y_test)
                precision[0,i]=m2.prec(x_test,y_test)
                recall[0,i]=m2.recall(x_test,y_test)
                f1[0,i]=m2.F1(x_test,y_test)
                time_model[0,i]= stop-start

               
                
            e = np.mean(error, axis=1)
            a = np.mean(accuracy, axis=1)
            p = np.mean(precision, axis=1)
            rec = np.mean(recall, axis=1)
            f = np.mean(f1, axis=1)
            print('TriTraining Disagree test error=', e)
            print('TriTraining Disagree test accuracy=', a)
            print('TriTraining Disagree test precision=', p)
            print('TriTraining Disagree test recall score=', rec)
            print('TriTraining Disagree test F1 score=', f)
            print('Training time=', np.mean(time_model, axis=1))
            print("\n")

classifier: DecisionTree
label_rate: 0.2
TriTraining Disagree test error= [0.]
TriTraining Disagree test accuracy= [1.]
TriTraining Disagree test precision= [1.]
TriTraining Disagree test recall score= [1.]
TriTraining Disagree test F1 score= [1.]
Training time= [4.35212173]


classifier: BP_Network
label_rate: 0.2
TriTraining Disagree test error= [0.19233792]
TriTraining Disagree test accuracy= [0.80766208]
TriTraining Disagree test precision= [1.]
TriTraining Disagree test recall score= [0.61509725]
TriTraining Disagree test F1 score= [0.76168447]
Training time= [10.53135269]


classifier: NaiveBayes
label_rate: 0.2
TriTraining Disagree test error= [3.77888961e-07]
TriTraining Disagree test accuracy= [0.99999962]
TriTraining Disagree test precision= [1.]
TriTraining Disagree test recall score= [0.99999924]
TriTraining Disagree test F1 score= [0.99999962]
Training time= [11.38365376]


